In [1]:
%load_ext nb_black
# https://medium.com/openplanetary/code-formatting-in-jupyter-cells-8fee4eda072f

<IPython.core.display.Javascript object>

## Prepare Train, Test and Validation set

In [8]:
import os
import pandas as pd
import numpy as np
import json
import mmcv
import itertools
import shutil
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import sys
import re

from collections import defaultdict
from PIL import Image
from IPython.display import display, HTML

sys.path.append("/home/badhon/Documents/thesis/thesis-codes/")
from classes.WheatDataset import WheatDataset
from helpers import utils

%matplotlib inline

<IPython.core.display.Javascript object>

#### Define some paths

In [4]:
path_splits = "/home/badhon/Documents/thesis/AerialDetection/data/wheat_kaggle/"
path_train_images = path_splits + "train/images/"
path_test_images = path_splits + "test/images/"

path_csv = path_splits + "train.csv"

<IPython.core.display.Javascript object>

#### Generate train.json file

In [6]:
train_df = pd.read_csv(path_csv)

<IPython.core.display.Javascript object>

In [9]:
train_df["x"] = -1
train_df["y"] = -1
train_df["w"] = -1
train_df["h"] = -1


def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r


train_df[["x", "y", "w", "h"]] = np.stack(
    train_df["bbox"].apply(lambda x: expand_bbox(x))
)
train_df.drop(columns=["bbox"], inplace=True)
train_df["x"] = train_df["x"].astype(np.float)
train_df["y"] = train_df["y"].astype(np.float)
train_df["w"] = train_df["w"].astype(np.float)
train_df["h"] = train_df["h"].astype(np.float)

<IPython.core.display.Javascript object>

In [10]:
print(train_df.head())

    image_id  width  height   source      x      y      w      h
0  b6ab77fd7   1024    1024  usask_1  834.0  222.0   56.0   36.0
1  b6ab77fd7   1024    1024  usask_1  226.0  548.0  130.0   58.0
2  b6ab77fd7   1024    1024  usask_1  377.0  504.0   74.0  160.0
3  b6ab77fd7   1024    1024  usask_1  834.0   95.0  109.0  107.0
4  b6ab77fd7   1024    1024  usask_1   26.0  144.0  124.0  117.0


<IPython.core.display.Javascript object>

In [21]:
train_df_grouped = train_df.groupby("image_id")

<IPython.core.display.Javascript object>

In [22]:
def xywh_poly(xmin, ymin, w, h):
    xmax = xmin + w
    ymax = ymin + h
    return [xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax]

<IPython.core.display.Javascript object>

In [23]:
data_dict = {
    "images": [],
    "categories": [
        {
            "id": 1,
            "name": "wheat_head",
            "supercategory": "",
            "color": "#902398",
            "metadata": {},
        }
    ],
    "annotations": [],
}

image_id = 1
anno_id = 1
for group_name, df_group in train_df_grouped:
    data_dict["images"].append(
        {
            "id": image_id,
            "dataset_id": 1,
            "path": path_train_images + group_name + ".jpg",
            "width": 1024,
            "height": 1024,
            "file_name": group_name + ".jpg",
        }
    )
    for row_index, row in df_group.iterrows():
        data_dict["annotations"].append(
            {
                "id": anno_id,
                "image_id": image_id,
                "category_id": 1,
                "segmentations": [xywh_poly(row["x"], row["y"], row["w"], row["h"])],
                "area": row["w"] * row["h"],
                "bbox": [row["x"], row["y"], row["w"], row["h"]],
                "iscrowd": False,
                "isbbox": True,
                "color": "#902398",
                "metadata": {},
            }
        )

        anno_id = anno_id + 1

    image_id = image_id + 1

<IPython.core.display.Javascript object>

In [26]:
mmcv.dump(data_dict, os.path.join(path_splits, "train/train.json"))

<IPython.core.display.Javascript object>

#### Load dataset

In [27]:
train_dataset = WheatDataset(
    os.path.join(path_splits, "train/train.json"), path_train_images
)
train_dataset.load_dataset()

<IPython.core.display.Javascript object>

#### Display Splits

In [28]:
display(train_dataset.get_dataset_stat())

,filename,no_of_anno
0,00333207f.jpg,55
1,005b0d8bb.jpg,20
2,006a994f7.jpg,25
3,00764ad5d.jpg,41
4,00b5fefed.jpg,25
...,...,...
3368,ffb445410.jpg,57
3369,ffbf75e5b.jpg,52
3370,ffbfe7cc0.jpg,34
3371,ffc870198.jpg,41


<IPython.core.display.Javascript object>